In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, (2, 2))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, (2, 2))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        '''
        This function enables to use any input dimentsion as it 
        automatically parses the size and flattens the layer.
        '''
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()
print(net)

Net (
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear (400 -> 120)
  (fc2): Linear (120 -> 84)
  (fc3): Linear (84 -> 10)
)


In [3]:
params = list(net.parameters())
print len(params)
print params[0].size()

10
torch.Size([6, 1, 5, 5])


In [5]:
input = Variable(torch.randn(1, 1, 32, 32))
out = net(input)
print out


 0.0962  0.0984  0.0275 -0.1353 -0.0008 -0.0040  0.1414 -0.0697 -0.0894 -0.0338
[torch.FloatTensor of size 1x10]



In [7]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [8]:
output = net(input)
target = Variable(torch.arange(1, 11))
criterion = nn.MSELoss()

loss = criterion(output, target)
print loss

Variable containing:
 38.6883
[torch.FloatTensor of size 1]



In [10]:
net.zero_grad()

print net.conv1.bias.grad

loss.backward()

print net.conv1.bias.grad

Variable containing:
 0
 0
 0
 0
 0
 0
[torch.FloatTensor of size 6]

Variable containing:
-0.1698
-0.1005
 0.1102
-0.0094
 0.0778
 0.0654
[torch.FloatTensor of size 6]



In [11]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr = 0.01)
optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()